<a href="https://colab.research.google.com/github/Ferdousi24/project_5803/blob/main/project_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openeo
!pip install matplotlib
!pip install xarray
!pip install rioxarray
!pip install rasterio
!pip install geopandas



In [ ]:
import openeo
import json
import xarray as xr
import rioxarray as rio

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd

from mpl_toolkits.axes_grid1 import make_axes_locatable

import rasterio
from rasterio.plot import show
from rasterio.mask import mask






In [ ]:
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

In [ ]:

# Bounding box coordinates
bounding_box = [44.96, -76.35, 45.53, -75.24]

coordinates = [
    [
        [bounding_box[1], bounding_box[0]],
        [bounding_box[1], bounding_box[2]],
        [bounding_box[3], bounding_box[2]],
        [bounding_box[3], bounding_box[0]],
        [bounding_box[1], bounding_box[0]]
    ]
]

aoi = {
    "type": "Polygon",
    "coordinates": coordinates
}





In [ ]:
# Bounding box coordinates
bounding_box = [44.96, -76.35, 45.53, -75.24]

# Constructing spatial extent
spatial_extent = {
    "west": bounding_box[1],
    "south": bounding_box[0],
    "east": bounding_box[3],
    "north": bounding_box[2]
}



In [ ]:
methane = connection.load_collection(
    "SENTINEL_5P_L2",
    temporal_extent=["2023-03-01", "2024-03-31"],
    spatial_extent={'west': -76.35, 'south': 44.96, 'east': -75.24, 'north': 45.53},
    bands=["CH4"],
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#methane.download("/content/drive/MyDrive/R-CH4.nc")

In [ ]:
ncfile = xr.open_dataset("/content/drive/MyDrive/R-CH4.nc", decode_coords="all")



In [ ]:
daily_average = ncfile['CH4'].mean(dim='t')


In [ ]:

# Calculate statistics
mean_value = daily_average.mean().values
median_value = np.median(daily_average.values)
min_value = daily_average.min().values
max_value = daily_average.max().values

# Visualize distribution with a histogram
plt.hist(daily_average.values.flatten(), bins=50, color='blue', alpha=0.7)
plt.axvline(x=mean_value, color='red', linestyle='--', label=f'Mean: {mean_value:.2f}')
plt.axvline(x=median_value, color='green', linestyle='--', label=f'Median: {median_value:.2f}')
plt.xlabel('Methane Concentration (ppb)')
plt.ylabel('Frequency')
plt.title('Distribution of Daily Average Methane Concentration')
plt.legend()
plt.show()

vmin = min_value
vmax = max_value


In [ ]:

cmmap = plt.cm.get_cmap("jet")
cmmap.set_bad('dimgrey', 1.)

# Displaying the variable 'z' i.e., methane concentration
fig, ax = plt.subplots()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)

# Plot the methane concentration data
im = ax.imshow(asi_mt, cmap=cmmap, vmin=1800, vmax=2000)  #  vmin and vmax according to data

# Add colorbar
fig.colorbar(im, cax=cax, orientation='vertical', label='CH4 Concentration (ppb)')

# Hide x and y axes
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

# Set plot size and title
plt.gcf().set_size_inches(15, 15)
ax.set_title('Daily Average Methane Concentration', fontsize=20)

# Set DPI (dots per inch) for the figure
fig.set_dpi(300.0)

# Show the plot
plt.show()

In [ ]:
# Calculate monthly average
monthly_average = ncfile['CH4'].resample(t='1M').mean(dim='t', skipna=True)
average_final = np.nanmean(monthly_average, axis=(1, 2))


time_values = monthly_average['t'].values

time_index = pd.to_datetime(time_values)

# Plot the time series
plt.plot(time_index, average_final, marker='o', linestyle='-')
plt.xlabel('Time')
plt.ylabel('Average Methane Concentration (ppb)')
plt.title('Time Series of Average Methane Concentration over AOI')

# Format x-axis to display month names
plt.xticks(time_index, [x.strftime('%b\n%Y') for x in time_index], rotation=45, ha='right')

plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

output_dir = "/content/drive/MyDrive/"
daily_average.to_netcdf(output_dir + "daily_average.nc")

dncfile = xr.open_dataset('/content/drive/MyDrive//daily_average.nc')
asi_mt=dncfile.variables['CH4'][:]

In [ ]:
#Open the daily_average nc file
dncfile = xr.open_dataset('/content/drive/MyDrive//daily_average.nc')
#print(dncfile)
#CH4 variables
mt=dncfile.variables['CH4'][:]




In [ ]:
#Checking the logitude range
longitude = dncfile['x']

# Check the range of longitude values
min_longitude = longitude.min().item()
max_longitude = longitude.max().item()

# Determine if the range is within 180 or 360
if min_longitude >= 0 and max_longitude <= 360:
    print("Longitude data is in the range of 0-360 degrees.")
elif min_longitude >= -180 and max_longitude <= 180:
    print("Longitude data is in the range of +/- 180 degrees.")
else:
    print("Longitude data does not match the expected range.")

In [ ]:
#Reading the shapefile
s_aoi = gpd.read_file("/Ottawa_Neighbourhood_Study_(ONS)_-_Neighbourhood_Boundaries_Gen_2.shp")
print(s_aoi.crs)
print(s_aoi.total_bounds)



In [ ]:
# transform the crs according to shapefile
mt = mt.rio.set_spatial_dims('x', 'y')

mt.rio.set_crs("epsg:4326")
#mt.rio.write_nodata(-9999)
print(mt.rio.crs)

mt.rio.to_raster(r"GeoTIFF.tif")

In [ ]:
# open the geotiff file and check properties
ras=rasterio.open('/content/GeoTIFF.tif')
ras.meta
print(ras.meta)

In [ ]:
#plotting the shapefile and raster
fig, ax = plt.subplots(1, 2)

# Show the raster
show(ras, ax=ax[0], title="Methane Concentration")

# Plot the shapefile
s_aoi.plot(ax=ax[1], facecolor='none', edgecolor='black')
#showing the raster and shapefile
plt.show()

In [ ]:

# Clip the raster to the extent of the shapefile
ras_clipped, ras_transform = mask(rasterio.open('/content/GeoTIFF.tif'), s_aoi.geometry, crop=True)

fig, ax = plt.subplots(figsize=(10, 5))

# Show the clipped raster
show(ras_clipped, ax=ax, title="Methane")

# Show the plot
plt.show()
